# Assignment 2

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
from patsy import dmatrices
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [9]:
# DATA IMPORT - FROM GITHUB
data_raw = pd.read_csv('https://raw.githubusercontent.com/thimaipham/2.CEU_DA3_Assignment2/main/listings.csv')

In [19]:
# Removing unnecessary columns to streamline the dataset for this assignment
# List of columns to be dropped based on the description and analysis
columns_to_drop = [
    'id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
    'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 
    'host_thumbnail_url', 'host_picture_url', 'host_about', 'host_neighbourhood',
    'host_listings_count', 'host_total_listings_count', 'neighbourhood', 
    'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'calendar_last_scraped', 
    'license', 'host_location', 'source','first_review','last_review','host_verifications',
    'calendar_updated', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
    'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'
]


# Dropping the columns from the DataFrame
data_cleaned = data_raw.drop(columns=columns_to_drop, errors='ignore')

# Display the shape of the new DataFrame to confirm the drop
data_cleaned.info


<bound method DataFrame.info of        host_since host_response_time host_response_rate host_acceptance_rate  \
0      2016-02-20                NaN                NaN                  NaN   
1      2015-07-22     within an hour               100%                 100%   
2      2014-06-04     within an hour               100%                  51%   
3      2016-08-26     within an hour                99%                 100%   
4      2013-09-03                NaN                NaN                 100%   
...           ...                ...                ...                  ...   
25475  2022-10-21     within an hour               100%                  99%   
25476  2013-07-10     within an hour               100%                 100%   
25477  2016-07-22     within an hour               100%                 100%   
25478  2015-07-13     within an hour               100%                  50%   
25479  2013-02-13     within an hour               100%                 100%   

      h

In [23]:
data_cleaned.head().T

,0,1,2,3,4
host_since,2016-02-20,2015-07-22,2014-06-04,2016-08-26,2013-09-03
host_response_time,NaN,within an hour,within an hour,within an hour,NaN
host_response_rate,NaN,100%,100%,99%,NaN
host_acceptance_rate,NaN,100%,51%,100%,100%
host_is_superhost,f,t,f,f,f
host_has_profile_pic,t,t,t,t,t
host_identity_verified,t,t,t,t,f
latitude,-33.77872,-34.08265,-33.83407,-33.867527,-33.6319
longitude,150.92234,151.14855,151.23442,151.084352,151.32902
property_type,Camper/RV,Private room in guesthouse,Entire home,Entire rental unit,Entire home
